# Accessing the Covetly MongoDB database

In this notebook, we access the wishlist and collection data we need for the recommender system.  The data is aggregated from the MongoDB database at Covetly.

In [1]:
from pymongo import MongoClient
from pprint import pprint
import operator
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# load databases
db = open('db.txt', 'r').readline().split(',')
domainDb = MongoClient(db[0], readPreference="secondary")[db[1]]
catalogDb = MongoClient(db[2], readPreference="secondary")[db[3]]

## MongoDB Aggregation
Here we extract the collection and wishlist data for all users.  To do this, we create an aggregation query.  The query takes some time to execute, so we split it into manageable chunks.  [truncated for brevity]

In [3]:
skipsize = 10000

# wishlists
wantitems = domainDb["wantlistitems"]
nwantitems = wantitems.count()

# collections
haveitems = domainDb["collectionitems"]
nhaveitems = haveitems.count()

# get wishlists
print('generating want list', nwantitems, 'entries')
n = 0
want_list = []
while n*skipsize <= nwantitems:
    print('... processing [', n*skipsize, ',', (n+1)*skipsize, ']')
    pipeline = [
        {"$skip": n*skipsize},
        {"$limit": skipsize},
        {"$group": {"_id": "$UserId", "items": {"$push": "$ItemId"}, "dates": {"$push":"$CreatedDate" }}}
    ]
    want_list.extend(list(wantitems.aggregate(pipeline)))
    n += 1
print('done!')

# get collections
print('generating have list: ', nhaveitems, 'entries')
n = 0
have_list = []
while n*skipsize <= nhaveitems:
    print('... processing [', n*skipsize, ',', (n+1)*skipsize, ']')
    pipeline = [
        {"$skip": n*skipsize},
        {"$limit": skipsize},
        {"$group": {"_id": "$UserId", "items": {"$push": "$ItemId"}, "dates": {"$push":"$CreatedDate" }}}
    ]
    have_list.extend(list(haveitems.aggregate(pipeline)))
    n += 1
print('done!')

generating want list 557582 entries
... processing [ 0 , 10000 ]
... processing [ 10000 , 20000 ]
... processing [ 20000 , 30000 ]
... processing [ 30000 , 40000 ]
... processing [ 40000 , 50000 ]
done!
generating have list:  1499952 entries
... processing [ 0 , 10000 ]
... processing [ 10000 , 20000 ]
... processing [ 20000 , 30000 ]
... processing [ 30000 , 40000 ]
... processing [ 40000 , 50000 ]
done!


## Combining collection and wishlist data
The collection and wishlist data will be stored in separate files, but will eventually go into a single ratings matrix.  We need to make sure that we use the same indices and columns for both.

[The database queries have been truncated above, so we are seeing only a fraction of the users and items.]

In [4]:
# find all users
users = set()
for w in want_list:
    users.add(w['_id'])
for h in have_list:
    users.add(h['_id'])
print('users: ', len(users))

# find all items
items = set()
for w in want_list:
    for i in w['items']:
        items.add(i)
for h in have_list:
    for i in h['items']:
        items.add(i)
print('items: ', len(items))

users:  1543
items:  4616


## Export to CSV
We save the data we need in csv files.

In [5]:
df_wantitems = pd.DataFrame(index=users, columns=items)

# populate df
for w in want_list:
    user = w['_id']
    for i in range(len(w['items'])):
        #df_wantitems.set_value(user, w['items'][i], w['dates'][i])
        df_wantitems.set_value(user, w['items'][i], 1)#w['dates'][i])
df_wantitems.fillna(0, inplace=True)

# export to csv
df_wantitems.to_csv('data/wishlist.csv')

In [6]:
df_haveitems = pd.DataFrame(index=users, columns=items)

# populate df
for h in have_list:
    user = h['_id']
    for i in range(len(h['items'])):
        #df_haveitems.set_value(user, h['items'][i], h['dates'][i])
        df_haveitems.set_value(user, h['items'][i], 1)#h['dates'][i]
df_haveitems.fillna(0, inplace=True)

# export to csv
df_haveitems.to_csv('data/havelist.csv')